# Julia 深度學習：類神經網路模型簡介

## 作業 032：訓練 MLP 學習門牌號碼資料集

訓練一個 MLP 模型來學習門牌號碼資料集。

注意：MLP 模型的能力有限，可能會導致訓練起來效果不佳。

注意：近期 Flux 正在持續更新，請確保您的 Julia 在 v1.3 版以上，以及 Flux 在 v0.10.4 以上或是最新版。

In [11]:
using Pkg
#Pkg.add("Flux")
Pkg.update()

   Updating registry at `C:\Users\chris-wang\.julia\registries\General`


   Updating git-repo `https://github.com/JuliaRegistries/General.git`


  Installed CUDAapi ────── v4.0.0
  Installed Flux ───────── v0.10.4
  Installed Plots ──────── v1.2.4
  Installed GPUArrays ──── v3.3.0
  Installed CUDAdrv ────── v6.3.0
  Installed CuArrays ───── v2.2.0
  Installed ColorTypes ─── v0.10.3
  Installed Cthulhu ────── v1.0.2
  Installed CodeTracking ─ v0.5.11
  Installed CUDAnative ─── v3.1.0
  Installed GPUCompiler ── v0.2.0
  Installed ColorSchemes ─ v3.7.0
  Installed GR ─────────── v0.49.1
   Updating `C:\Users\chris-wang\.julia\environments\v1.4\Project.toml`
  [587475ba] ↑ Flux v0.10.0 ⇒ v0.10.4
  [91a5bcdd] ↑ Plots v1.0.14 ⇒ v1.2.4
   Updating `C:\Users\chris-wang\.julia\environments\v1.4\Manifest.toml`
  [1520ce14] ↑ AbstractTrees v0.2.1 ⇒ v0.3.3
  [fa961155] ↑ CEnum v0.2.0 ⇒ v0.3.0
  [3895d2a7] ↑ CUDAapi v3.1.0 ⇒ v4.0.0
  [c5f51814] ↑ CUDAdrv v6.0.0 ⇒ v6.3.0
  [be33ccc6] ↑ CUDAnative v2.10.2 ⇒ v3.1.0
  [da1fd8a2] + CodeTracking v0.5.11
  [944b1d66] ↑ CodecZlib v0.6.0 ⇒ v0.7.0
  [35d6a980] + ColorSchemes v3.7.0
  [3da002f7] ↑ Col

In [1]:
using Flux
using Flux.Data: DataLoader
using Flux: @epochs, onecold, onehotbatch, throttle, logitcrossentropy
using MLDatasets
using Statistics

## 讀取資料

In [2]:
train_X, train_y = SVHN2.traindata(Float32)
test_X,  test_y  = SVHN2.testdata(Float32)

(Float32[0.14901961 0.15294118 … 0.19607843 0.1882353; 0.15294118 0.15294118 … 0.2 0.1882353; … ; 0.16470589 0.16862746 … 0.1764706 0.17254902; 0.15294118 0.15294118 … 0.16470589 0.16470589]

Float32[0.40392157 0.40784314 … 0.45882353 0.4509804; 0.40784314 0.40784314 … 0.4627451 0.4509804; … ; 0.40392157 0.39607844 … 0.45490196 0.4509804; 0.38039216 0.38039216 … 0.44313726 0.44313726]

Float32[0.23529412 0.23921569 … 0.29803923 0.2901961; 0.23921569 0.23921569 … 0.3019608 0.2901961; … ; 0.24313726 0.24705882 … 0.28235295 0.2784314; 0.22352941 0.22352941 … 0.27058825 0.2784314]

Float32[0.5058824 0.5254902 … 0.5411765 0.5137255; 0.49803922 0.52156866 … 0.50980395 0.47843137; … ; 0.48235294 0.49411765 … 0.39607844 0.43529412; 0.48235294 0.49019608 … 0.4392157 0.48235294]

Float32[0.5568628 0.5882353 … 0.59607846 0.5686275; 0.56078434 0.58431375 … 0.5647059 0.53333336; … ; 0.5254902 0.5372549 … 0.41960785 0.4627451; 0.5294118 0.5372549 … 0.4627451 0.50980395]

Float32[0.6 0.627451 … 0.647

In [97]:
train_X

32×32×3×73257 Array{Float32,4}:
[:, :, 1, 1] =
 0.129412   0.109804  0.156863   0.188235   …  0.337255   0.329412   0.333333
 0.0588235  0.054902  0.0705882  0.0745098     0.341176   0.337255   0.34902
 0.0588235  0.054902  0.0627451  0.0901961     0.321569   0.321569   0.333333
 0.164706   0.168627  0.117647   0.0901961     0.313726   0.305882   0.333333
 0.262745   0.270588  0.25098    0.231373      0.243137   0.317647   0.317647
 0.266667   0.231373  0.227451   0.262745   …  0.0823529  0.180392   0.282353
 0.266667   0.231373  0.243137   0.239216      0.0784314  0.0901961  0.121569
 0.290196   0.298039  0.301961   0.305882      0.341176   0.352941   0.352941
 0.305882   0.32549   0.329412   0.34902       0.419608   0.427451   0.439216
 0.345098   0.345098  0.34902    0.352941      0.411765   0.403922   0.415686
 0.329412   0.321569  0.32549    0.329412   …  0.415686   0.4        0.396078
 0.309804   0.329412  0.32549    0.333333      0.411765   0.407843   0.423529
 0.309804   0.3294

呼叫 SVHN2 資料集的過程中，會先去檢查以前是否有下載過，如果是第一次下載，過程中會詢問是否下載資料集，請回答 `y`。整個資料集的大小約為 1.6 GB，下載時間可能會稍久，請耐心等候。

In [3]:
#train_X = Flux.flatten(train_X)
#test_X = Flux.flatten(test_X)
train_y = onehotbatch(train_y, 1:10)
test_y = onehotbatch(test_y, 1:10)

10×26032 Flux.OneHotMatrix{Array{Flux.OneHotVector,1}}:
 0  0  1  0  0  1  0  1  1  0  0  0  0  …  1  0  1  0  1  0  0  0  0  0  0  0
 0  1  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  1  1  0  0  0
 0  0  0  0  0  0  0  0  0  0  1  0  0     0  0  0  0  0  1  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 1  0  0  0  0  0  0  0  0  0  0  0  1     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  1  0  0  0  0  0  0  1  0  …  0  0  0  0  0  0  1  0  0  0  1  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  1  0  0  0  0  0  1  0  1
 0  0  0  0  0  0  0  0  0  1  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  1  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  1  0  0  0  0  0  0  0  0  0     0  1  0  0  0  0  0  0  0  0  0  0

In [49]:
batchsize=1024 
train= DataLoader(train_X, train_y, batchsize=batchsize, shuffle=true)
test= DataLoader(test_X, test_y, batchsize=batchsize);

In [82]:
loss(x, y)= crossentropy(model(x), y)

loss (generic function with 1 method)

## callbacks函式

In [51]:
function test_loss()
    l=0f0
    for (x, y) in test
        l += loss(x, y)
    end
    l/length(test)
end

test_loss (generic function with 1 method)

In [85]:
#evalcb()= @show(test_loss())
evalcb() = @show(accuracy(test_X, test_y))

evalcb (generic function with 1 method)

## 模型訓練

In [5]:
using Metalhead
using Metalhead: classify

┌ Info: Precompiling Metalhead [dbeba491-748d-5e0e-a39e-b530a07fa0cc]
└ @ Base loading.jl:1260


In [6]:
vgg=VGG19()
length(vgg.layers)
model=Chain(vgg.layers[1:21],flatten,Dense(512,4096,relu),Dropout(0.5),Dense(4096,4096,relu),Dropout(0.5),Dense(4096,10),softmax)

Chain(Chain(Conv((3, 3), 3=>64, relu), Conv((3, 3), 64=>64, relu), MaxPool((2, 2), pad = (0, 0, 0, 0), stride = (2, 2)), Conv((3, 3), 64=>128, relu), Conv((3, 3), 128=>128, relu), MaxPool((2, 2), pad = (0, 0, 0, 0), stride = (2, 2)), Conv((3, 3), 128=>256, relu), Conv((3, 3), 256=>256, relu), Conv((3, 3), 256=>256, relu), Conv((3, 3), 256=>256, relu), MaxPool((2, 2), pad = (0, 0, 0, 0), stride = (2, 2)), Conv((3, 3), 256=>512, relu), Conv((3, 3), 512=>512, relu), Conv((3, 3), 512=>512, relu), Conv((3, 3), 512=>512, relu), MaxPool((2, 2), pad = (0, 0, 0, 0), stride = (2, 2)), Conv((3, 3), 512=>512, relu), Conv((3, 3), 512=>512, relu), Conv((3, 3), 512=>512, relu), Conv((3, 3), 512=>512, relu), MaxPool((2, 2), pad = (0, 0, 0, 0), stride = (2, 2))), flatten, Dense(512, 4096, relu), Dropout(0.5), Dense(4096, 4096, relu), Dropout(0.5), Dense(4096, 10), softmax)

In [101]:
epochs=20
@epochs epochs Flux.train!(loss, params(model), train, ADAM(0.005),cb=throttle(evalcb, 10))


┌ Info: Epoch 1
└ @ Main C:\Users\chris-wang\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121


DimensionMismatch: DimensionMismatch("A has dimensions (4096,25088) but B has dimensions (512,1024)")

In [24]:
accuracy(x, y)= mean(onecold(model(x)).==onecold(y))

accuracy (generic function with 1 method)

In [25]:
accuracy(test_X,test_y)

0.1958743085433313

In [89]:
using Flux, Metalhead, Statistics
using Flux: onehotbatch, onecold, crossentropy, throttle
using Metalhead: trainimgs
using Images: channelview
using Statistics: mean
using Base.Iterators: partition

In [90]:
X = trainimgs(CIFAR10)
imgs = [getarray(X[i].img) for i in 1:50000]
labels = onehotbatch([X[i].ground_truth.class for i in 1:50000],1:10)
train = gpu.([(cat(imgs[i]..., dims = 4), labels[:,i]) for i in partition(1:49000, 100)])
valset = collect(49001:50000)
valX = cat(imgs[valset]..., dims = 4) |> gpu
valY = labels[:, valset] |> gpu

UndefVarError: UndefVarError: CIFAR10 not defined

In [ ]:
vgg19() = Chain(
  Conv((3, 3), 3 => 64, relu, pad=(1, 1), stride=(1, 1)),
  BatchNorm(64),
  Conv((3, 3), 64 => 64, relu, pad=(1, 1), stride=(1, 1)),
  BatchNorm(64),
  x -> maxpool(x, (2, 2)),
  Conv((3, 3), 64 => 128, relu, pad=(1, 1), stride=(1, 1)),
  BatchNorm(128),
  Conv((3, 3), 128 => 128, relu, pad=(1, 1), stride=(1, 1)),
  BatchNorm(128),
  x -> maxpool(x, (2, 2)),
  Conv((3, 3), 128 => 256, relu, pad=(1, 1), stride=(1, 1)),
  BatchNorm(256),
  Conv((3, 3), 256 => 256, relu, pad=(1, 1), stride=(1, 1)),
  BatchNorm(256),
  Conv((3, 3), 256 => 256, relu, pad=(1, 1), stride=(1, 1)),
  BatchNorm(256),
  Conv((3, 3), 256 => 256, relu, pad=(1, 1), stride=(1, 1)),
  x -> maxpool(x, (2, 2)),
  Conv((3, 3), 256 => 512, relu, pad=(1, 1), stride=(1, 1)),
  BatchNorm(512),
  Conv((3, 3), 512 => 512, relu, pad=(1, 1), stride=(1, 1)),
  BatchNorm(512),
  Conv((3, 3), 512 => 512, relu, pad=(1, 1), stride=(1, 1)),
  BatchNorm(512),
  Conv((3, 3), 512 => 512, relu, pad=(1, 1), stride=(1, 1)),
  x -> maxpool(x, (2, 2)),
  Conv((3, 3), 512 => 512, relu, pad=(1, 1), stride=(1, 1)),
  BatchNorm(512),
  Conv((3, 3), 512 => 512, relu, pad=(1, 1), stride=(1, 1)),
  BatchNorm(512),
  Conv((3, 3), 512 => 512, relu, pad=(1, 1), stride=(1, 1)),
  BatchNorm(512),
  Conv((3, 3), 512 => 512, relu, pad=(1, 1), stride=(1, 1)),
  x -> maxpool(x, (2, 2)),
  x -> reshape(x, :, size(x, 4)),
  Dense(512, 4096, relu),
  Dropout(0.5),
  Dense(4096, 4096, relu),
  Dropout(0.5),
  Dense(4096, 10),
  softmax) |> gpu